In [ ]:
from collections import OrderedDict

import heapq

"""
请你设计并实现一个满足  LRU (最近最少使用) 缓存 约束的数据结构。
实现 LRUCache 类：
LRUCache(int capacity) 以 正整数 作为容量 capacity 初始化 LRU 缓存
int get(int key) 如果关键字 key 存在于缓存中，则返回关键字的值，否则返回 -1 。
void put(int key, int value) 如果关键字 key 已经存在，则变更其数据值 value ；如果不存在，则向缓存中插入该组 key-value 。如果插入操作导致关键字数量超过 capacity ，则应该 逐出 最久未使用的关键字。
函数get和put必须以 O(1) 的平均时间复杂度运行。
 
示例：

输入
["LRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]
[[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]
输出
[null, null, null, 1, null, -1, null, -1, 3, 4]

解释
LRUCache lRUCache = new LRUCache(2);
lRUCache.put(1, 1); // 缓存是 {1=1}
lRUCache.put(2, 2); // 缓存是 {1=1, 2=2}
lRUCache.get(1);    // 返回 1
lRUCache.put(3, 3); // 该操作会使得关键字 2 作废，缓存是 {1=1, 3=3}
lRUCache.get(2);    // 返回 -1 (未找到)
lRUCache.put(4, 4); // 该操作会使得关键字 1 作废，缓存是 {4=4, 3=3}
lRUCache.get(1);    // 返回 -1 (未找到)
lRUCache.get(3);    // 返回 3
lRUCache.get(4);    // 返回 4
 
提示：
1 <= capacity <= 3000
0 <= key <= 10000
0 <= value <= 105
最多调用 2 * 105 次 get 和 put
"""


class LRUCache:
    """
    性能问题
        min(self.time_dict, key=self.time_dict.get)：
        
        每次插入时，都需要遍历 time_dict 找到最久未使用的键。
        
        时间复杂度为 O(n)，其中 n 是缓存容量。对于大容量缓存，性能会很差。
    
    时间戳溢出
        self.time 是一个整数，随着时间推移会不断增加。
        
        如果程序运行时间很长，self.time 可能会溢出。
    
    冗余存储
        dict 和 time_dict 分别存储键值对和最后访问时间，导致额外的空间开销。
        
    """
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.time = 0
        self.dict = {}
        self.time_dict = {}

    def get(self, key: int) -> int:
        if key in self.dict:
            self.time_dict[key] = self.time
            self.time += 1
            return self.dict[key]
        return -1

    def put(self, key: int, value: int) -> None:
        self.dict[key] = value
        self.time_dict[key] = self.time
        self.time += 1
        if len(self.time_dict) > self.capacity:
            oldest = min(self.time_dict, key=self.time_dict.get)
            del self.dict[oldest]
            del self.time_dict[oldest]

class LRUCache2:
    """
    优化建议
        使用 OrderedDict
        Python 的 collections.OrderedDict 可以维护键的插入顺序，非常适合实现 LRU 缓存。
    """
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = OrderedDict()

    def get(self, key: int) -> int:
        if key in self.cache:
            # 将访问的键移到末尾（表示最近使用）
            self.cache.move_to_end(key)
            return self.cache[key]
        return -1

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            # 如果键已存在，更新值并移到末尾
            self.cache.move_to_end(key)
        self.cache[key] = value  # 插入或更新键值对
        if len(self.cache) > self.capacity:
            # 删除最久未使用的键（位于 OrderedDict 的开头）
            self.cache.popitem(last=False)
            
class LRUCache3:
    """
    核心思想
        LRU 缓存的核心思想是：
        
            当缓存满时，淘汰最久未使用的数据。
            
            每次访问或插入数据时，将该数据标记为最近使用。
        
        为了实现 O(1) 的时间复杂度，我们需要：
        
            快速查找：使用哈希表（dict）来存储键值对，实现 O(1) 的查找。
            
            快速维护顺序：使用双向链表来维护键的访问顺序，链表头部是最久未使用的数据，链表尾部是最近使用的数据。
    """
    class DLinkedNode:
        def __init__(self, key=0, value=0):
            self.key = key
            self.value = value
            self.prev = None
            self.next = None
        
    def __init__(self, capacity: int):
        """
        LRU 缓存类需要维护以下信息：
            cache：哈希表，用于存储键和对应的节点。
            capacity：缓存的最大容量。
            head 和 tail：虚拟头节点和虚拟尾节点，用于简化链表操作。
        """
        self.capacity = capacity
        self.cache = {}
        self.head = self.DLinkedNode()  # 虚拟头节点
        self.tail = self.DLinkedNode()  # 虚拟尾节点
        self.head.next = self.tail
        self.tail.prev = self.head

    def get(self, key: int) -> int:
        """
        每次访问或插入节点时，需要将该节点移到链表尾部，表示最近使用。
        """
        if key in self.cache:
            node = self.cache[key]
            self._move_to_tail(node)  # 将节点移到链表尾部
            return node.value
        return -1

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            node = self.cache[key]
            node.value = value
            self._move_to_tail(node)  # 将节点移到链表尾部
        else:
            if len(self.cache) == self.capacity:
                removed = self._remove_head()  # 删除链表头部的节点
                del self.cache[removed.key]
            node = self.DLinkedNode(key, value)
            self.cache[key] = node
            self._add_to_tail(node)  # 将新节点添加到链表尾部

    def _move_to_tail(self, node):
        self._remove_node(node)
        self._add_to_tail(node)

    def _remove_node(self, node):
        """
        将节点从链表中移除。
        """
        node.prev.next = node.next
        node.next.prev = node.prev

    def _add_to_tail(self, node):
        """
        将节点添加到链表尾部。
        """
        node.prev = self.tail.prev
        node.next = self.tail
        self.tail.prev.next = node
        self.tail.prev = node

    def _remove_head(self):
        """
        当缓存满时，删除链表头部的节点（最久未使用的节点）。
        """
        node = self.head.next
        self._remove_node(node)
        return node
# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)